In [1]:
import ROOT
import numpy as np
import pandas as pd

Welcome to JupyROOT 6.24/04


In [5]:
binned_fit_results_filename = f'/work/halld/home/viducic/data/fit_params/pipkmks/cross_section_values.csv'

rdf = ROOT.RDF.MakeCsvDataFrame(binned_fit_results_filename)

In [6]:
print(rdf.GetColumnNames())

{ "mean", "mean_error", "width", "width_error", "chi2ndf", "ks_test", "yield", "yield_error", "acceptance", "acceptance_error", "cross_section", "cross_section_error", "t_bin_middle", "t_bin_width", "beam_energy" }


In [ ]:
# TODO: plot for different beam energies 